
# How to generate text: using different decoding methods for language generation with Transformers

(based on https://github.com/huggingface/blog/blob/main/notebooks/02_how_to_generate.ipynb)

### **Introduction**

In recent years, there has been an increasing interest in open-ended language generation thanks to the rise of large transformer-based language models trained on millions of webpages, such as OpenAI's famous [GPT2 model](https://openai.com/blog/better-language-models/). The results on conditioned open-ended language generation are impressive, e.g. [GPT2 on unicorns](https://openai.com/blog/better-language-models/#samples), [XLNet](https://medium.com/@amanrusia/xlnet-speaks-comparison-to-gpt-2-ea1a4e9ba39e), [Controlled language with CTRL](https://blog.einstein.ai/introducing-a-conditional-transformer-language-model-for-controllable-generation/). Besides the improved transformer architecture and massive unsupervised training data, **better decoding methods** have also played an important role.

This blog post gives a brief overview of different decoding strategies and more importantly shows how *you* can implement them with very little effort using the popular `transformers` library!

All of the following functionalities can be used for **auto-regressive** language generation ([here](http://jalammar.github.io/illustrated-gpt2/) a refresher). In short, *auto-regressive* language generation is based on the assumption that the probability distribution of a word sequence can be decomposed into the product of conditional next word distributions:
$$ P(w_{1:T} | W_0 ) = \prod_{t=1}^T P(w_{t} | w_{1: t-1}, W_0) \text{ ,with }  w_{1: 0} = \emptyset, $$

and $W_0$ being the initial *context* word sequence. The length $T$ of the word sequence is usually determined *on-the-fly* and corresponds to the timestep $t=T$ the EOS token is generated from $P(w_{t} | w_{1: t-1}, W_{0})$.


Auto-regressive language generation is now available for `GPT2`, `XLNet`, `OpenAi-GPT`, `CTRL`, `TransfoXL`, `XLM`, `Bart`, `T5` in both PyTorch and Tensorflow >= 2.0!

We will give a tour of the currently most prominent decoding methods, mainly *Greedy search*, *Beam search*, *Top-K sampling* and *Top-p sampling*.


Let's quickly install transformers and load the model.

In [5]:
#!pip install -q git+https://github.com/huggingface/transformers.git  # the bleeding edge version
#!pip install -q tensorflow==2.1
!pip install transformers==4.48.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 28.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.49.0
    Uninstalling transformers-4.49.0:
      Successfully uninstalled transformers-4.49.0


In [1]:
import torch

In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")

# add the EOS token as PAD token to avoid warnings
model = GPT2LMHeadModel.from_pretrained("gpt2-medium", pad_token_id=tokenizer.eos_token_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

### **Greedy Search**

Greedy search simply selects the word with the highest probability as its next word: $w_t = argmax_{w}P(w | w_{1:t-1})$ at each timestep $t$. The following sketch shows greedy search.

![Greedy Search](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/greedy_search.png)

Starting from the word $\text{"The"}$, the algorithm
greedily chooses the next word of highest probability $\text{"nice"}$ and so on, so that the final generated word sequence is $\text{"The", "nice", "woman"}$ having an overall probability of $0.5 \times 0.4 = 0.2$.

In the following we will generate word sequences using GPT2 on the context $(\text{"I", "enjoy", "walking", "with", "my", "cute", "dog"})$. Let's see how greedy search can be used in `transformers` as follows:

In [3]:
# encode context the generation is conditioned on
input_ids = tokenizer.encode('I enjoy walking with my cute dog', return_tensors='pt')

# generate text until the output length (which includes the context length) reaches 50
greedy_output = model.generate(input_ids, max_length=50)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I can do it with my cat. I'm not sure if I can do it with my cat.

I'm not sure if I can do it with my cat.


Alright! We have generated our first short text with GPT2 😊. The generated words following the context are reasonable, but the model quickly starts repeating itself! This is a very common problem in language generation in general and seems to be even more so in greedy and beam search - check out [Vijayakumar et al., 2016](https://arxiv.org/abs/1610.02424) and [Shao et al., 2017](https://arxiv.org/abs/1701.03185).

The major drawback of greedy search though is that it misses high probability words hidden behind a low probability word as can be seen in our sketch above:

The word $\text{"has"}$ with its high conditional probability of $0.9$ is hidden behind the word $\text{"dog"}$, which has only the second-highest conditional probability, so that greedy search misses the word sequence $\text{"The"}, \text{"dog"}, \text{"has"}$.

Thankfully, we have beam search to alleviate this problem!


### **Beam search**

Beam search reduces the risk of missing hidden high probability word sequences by keeping the most likely `num_beams` of hypotheses at each time step and eventually choosing the hypothesis that has the overall highest probability. Let's illustrate with `num_beams=2`:

![Beam search](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/beam_search.png)

At time step $1$, besides the most likely hypothesis $\text{"The", "woman"}$, beam search also keeps track of the second most likely one $\text{"The", "dog"}$. At time step $2$, beam search finds that the word sequence $\text{"The", "dog", "has"}$ has with $0.36$ a higher probability than $\text{"The", "nice", "woman"}$, which has $0.2$. Great, it has found the most likely word sequence in our toy example!

Beam search will always find an output sequence with higher probability than greedy search, but is not guaranteed to find the most likely output.

Let's see how beam search can be used in `transformers`. We set `num_beams > 1` and `early_stopping=True` so that generation is finished when all beam hypotheses reached the EOS token.

In [4]:
# activate beam search and early_stopping
beam_output = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I don't want to do it every day. I want to do it once in a while, but I don't want to do it every day. I want to do it once in a while,


While the result is arguably more fluent, the output still includes repetitions of the same word sequences.  
A simple remedy is to introduce *n-grams* (*a.k.a* word sequences of $n$ words) penalties as introduced by [Paulus et al. (2017)](https://arxiv.org/abs/1705.04304) and [Klein et al. (2017)](https://arxiv.org/abs/1701.02810). The most common *n-grams* penalty makes sure that no *n-gram* appears twice by manually setting the probability of next words that could create an already seen *n-gram* to $0$.

Let's try it out by setting `no_repeat_ngram_size=2` so that no *2-gram* appears twice:

In [5]:
# set no_repeat_ngram_size to 2
beam_output = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    no_repeat_ngram_size=2,
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I don't have the time to do it every day. So, I decided to make my own dog walker.

I started with a simple design. I wanted to create something that I could


Nice, that looks much better! We can see that the repetition does not appear anymore. Nevertheless, *n-gram* penalties have to be used with care. An article generated about the city *New York* should not use a *2-gram* penalty or otherwise, the name of the city would only appear once in the whole text!

Another important feature about beam search is that we can compare the top beams after generation and choose the generated beam that fits our purpose best.

In `transformers`, we simply set the parameter `num_return_sequences` to the number of highest scoring beams that should be returned. Make sure though that `num_return_sequences <= num_beams`!

In [6]:
# set return_num_sequences > 1
beam_outputs = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    no_repeat_ngram_size=2,
    num_return_sequences=5,
    early_stopping=True
)

# now we have 3 output sequences
print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}\n".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: I enjoy walking with my cute dog, but I don't have the time to do it every day. So, I decided to make my own dog walker.

I started with a simple design. I wanted to create something that I could

1: I enjoy walking with my cute dog, but I don't have the time to do it every day. So, I decided to make my own dog walker.

I started with a simple design. I wanted to create something that could be

2: I enjoy walking with my cute dog, but I don't have the time to do it every day. So, I decided to make my own dog walker.

I started with a simple design. I wanted to create something that would be

3: I enjoy walking with my cute dog, but I don't have the time to do it every day. So, I decided to make my own dog walker.

I started with a simple design. I wanted to create a device that would

4: I enjoy walking with my cute dog, but I don't have the time to do it every day. So, I dec

As can be seen, the five beam hypotheses are only marginally different to each other - which should not be too surprising when using only 5 beams.

## Diverse beam search

In [7]:
# set return_num_sequences > 1
beam_outputs = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    no_repeat_ngram_size=2,
    num_return_sequences=5,
    early_stopping=True,
    num_beam_groups=5, # this must be a divisor of num_beams
    diversity_penalty=1.0,
)

print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}\n".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: I enjoy walking with my cute dog and she loves it. She is very friendly and loves to play with me. I am very happy with the purchase and I will definitely be buying more of these.

Rated 5 out of 5 by Anonymous from

1: I enjoy walking with my cute dog, and I love to play with her. I'm not sure why I've been so reluctant to share this with you, but I think it's because I don't want to be judged for it.



2: I enjoy walking with my cute dog. I love to play with her. She is so cute and I am so happy to have her with me.

I love the fact that I can go to the park with the dog and she can play

3: I enjoy walking with my cute dog, but I'm not sure if I can do it with a dog that's so big. I've been thinking about it, and I think I'll try it.

I'm going to try to get

4: I enjoy walking with my cute dog. She is so cute and I love to see her happy. I have been to the vet twice and 

In open-ended generation, a couple of reasons have recently been brought forward why beam search might not be the best possible option:

- Beam search can work very well in tasks where the length of the desired generation is more or less predictable as in machine translation or summarization - see [Murray et al. (2018)](https://arxiv.org/abs/1808.10006) and [Yang et al. (2018)](https://arxiv.org/abs/1808.09582). But this is not the case for open-ended generation where the desired output length can vary greatly, e.g. dialog and story generation.

- We have seen that beam search heavily suffers from repetitive generation. This is especially hard to control with *n-gram*- or other penalties in story generation since finding a good trade-off between forced "no-repetition" and repeating cycles of identical *n-grams* requires a lot of finetuning.

- As argued in [Ari Holtzman et al. (2019)](https://arxiv.org/abs/1904.09751), high quality human language does not follow a distribution of high probability next words. In other words, as humans, we want generated text to surprise us and not to be boring/predictable. The authors show this nicely by plotting the probability, a model would give to human text vs. what beam search does.

![alt text](https://blog.fastforwardlabs.com/images/2019/05/Screen_Shot_2019_05_08_at_3_06_36_PM-1557342561886.png)


So let's stop being boring and introduce some randomness 🤪.

## **Sampling**

In its most basic form, sampling means randomly picking the next word $w_t$ according to its conditional probability distribution:

$$w_t \sim P(w|w_{1:t-1})$$

Taking the example from above, the following graphic visualizes language generation when sampling.

![vanilla_sampling](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/sampling_search.png)

It becomes obvious that language generation using sampling is not *deterministic* anymore. The word
$\text{"car"}$ is sampled from the conditioned probability distribution $P(w | \text{"The"})$, followed by sampling $\text{"drives"}$ from $P(w | \text{"The"}, \text{"car"})$.

In `transformers`, we set `do_sample=True` and deactivate *Top-K* sampling (more on this later) via `top_k=0`. In the following, we will fix `random_seed=0` for illustration purposes. Feel free to change the `random_seed` to play around with the model.


In [8]:
# set seed to reproduce results. Feel free to change the seed though to get different results
torch.random.manual_seed(1)

# activate sampling and deactivate top_k by setting top_k sampling to 0
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_k=0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog when the sun is out (Exact same thing when I am not wearing a bikini or sticky palm), so when we visited the beach that 95% of the activities were women-only. The beaches were safe enough


Interesting! The text seems alright - but when taking a closer look, it is not very coherent. the *3-grams* *new hand sense* and *local batte harness* are very weird and don't sound like they were written by a human. That is the big problem when sampling word sequences: The models often generate incoherent gibberish, *cf.* [Ari Holtzman et al. (2019)](https://arxiv.org/abs/1904.09751).

A trick is to make the distribution $P(w|w_{1:t-1})$ sharper (increasing the likelihood of high probability words and decreasing the likelihood of low probability words) by lowering the so-called `temperature` of the [softmax](https://en.wikipedia.org/wiki/Softmax_function#Smooth_arg_max).

$q = \frac{exp(z_i / T)}{\sum_j exp(z_j / T)}$

An illustration of applying temperature to our example from above could look as follows.

![top_p_sampling](https://github.com/patrickvonplaten/scientific_images/blob/master/sampling_search_with_temp.png?raw=true)

The conditional next word distribution of step $t=1$ becomes much sharper leaving almost no chance for word $\text{"car"}$ to be selected.


Let's see how we can cool down the distribution in the library by setting `temperature=0.7`:

In [9]:
# set seed to reproduce results. Feel free to change the seed though to get different results
torch.random.manual_seed(0)

# use temperature to decrease the sensitivity to low probability candidates
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_k=0,
    temperature=0.7
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, so you can imagine how I must feel when I see these adorable puppies in my arms!


OK. There are less weird n-grams and the output is a bit more coherent now! While applying temperature can make a distribution less random, in its limit, when setting `temperature` $ \to 0$, temperature scaled sampling becomes equal to greedy decoding and will suffer from the same problems as before.



## **Top-K Sampling**

[Fan et. al (2018)](https://arxiv.org/pdf/1805.04833.pdf) introduced a simple, but very powerful sampling scheme, called ***Top-K*** sampling. In *Top-K* sampling, the *K* most likely next words are filtered and the probability mass is redistributed among only those *K* next words.
GPT2 adopted this sampling scheme, which was one of the reasons for its success in story generation.

We extend the range of words used for both sampling steps in the example above from 3 words to 10 words to better illustrate *Top-K* sampling.

![top_k_sampling](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/top_k_sampling.png)

Having set $K = 6$, in both sampling steps we limit our sampling pool to 6 words. While the 6 most likely words, defined as $V_{\text{top-K}}$ encompass only *ca.* two-thirds of the whole probability mass in the first step, it includes almost all of the probability mass in the second step. Nevertheless, we see that it successfully eliminates the rather weird candidates $\text{"not", "the", "small", "told"}$
in the second sampling step.


Let's see how *Top-K* can be used in the library by setting `top_k=50`:

In [10]:
# set seed to reproduce results. Feel free to change the seed though to get different results
torch.random.manual_seed(0)

# set top_k to 50
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_k=50
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog or riding my bike with my friends as long as I can avoid a traffic jam. However, when I'm driving, I'm in control. If I find myself in a ditch without a backup, or in a


Not bad at all! The text is arguably the most *human-sounding* text so far.
One concern though with *Top-K* sampling is that it does not dynamically adapt the number of words that are filtered from the next word probability distribution $P(w|w_{1:t-1})$.
This can be problematic as some words might be sampled from a very sharp distribution (distribution on the right in the graph above), whereas others from a much more flat distribution (distribution on the left in the graph above).

In step $t=1$, *Top-K* eliminates the possibility to
sample $\text{"people", "big", "house", "cat"}$, which seem like reasonable candidates. On the other hand, in step $t=2$ the method includes the arguably ill-fitted words $\text{"down", "a"}$ in the sample pool of words. Thus, limiting the sample pool to a fixed size *K* could endanger the model to produce gibberish for sharp distributions and limit the model's creativity for flat distribution.
This intuition led [Ari Holtzman et al. (2019)](https://arxiv.org/abs/1904.09751) to create ***Top-p***- or ***nucleus***-sampling.



## **Top-p (nucleus) sampling**

Instead of sampling only from the most likely *K* words, in *Top-p* sampling chooses from the smallest possible set of words whose cumulative probability exceeds the probability *p*. The probability mass is then redistributed among this set of words. This way, the size of the set of words (*a.k.a* the number of words in the set) can dynamically increase and decrease according to the next word's probability distribution. Ok, that was very wordy, let's visualize.

![top_p_sampling](https://github.com/patrickvonplaten/scientific_images/blob/master/top_p_sampling.png?raw=true)

Having set $p=0.92$, *Top-p* sampling picks the *minimum* number of words to exceed together $p=92\%$ of the probability mass, defined as $V_{\text{top-p}}$. In the first example, this included the 9 most likely words, whereas it only has to pick the top 3 words in the second example to exceed 92%. Quite simple actually! It can be seen that it keeps a wide range of words where the next word is arguably less predictable, *e.g.* $P(w | \text{"The"})$, and only a few words when the next word seems more predictable, *e.g.* $P(w | \text{"The", "car"})$.

Alright, time to check it out in `transformers`!
We activate *Top-p* sampling by setting `0 < top_p < 1`:

In [11]:
# set seed to reproduce results. Feel free to change the seed though to get different results
torch.random.manual_seed(3)

# deactivate top_k sampling and sample only from 92% most likely words
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_p=0.92,
    top_k=0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog Ethereal." The owner

Jillian said, "Our pet started showing us signs of autism and eventually we dropped him off. In all honesty, I think he loves her more and will give her love again


Great, that sounds like it could have been written by a human. Well, maybe not quite yet.

While in theory, *Top-p* seems more elegant than *Top-K*, both methods work  well in practice. *Top-p* can also be used in combination with *Top-K*, which can avoid very low ranked words while allowing for some dynamic selection.

Finally, to get multiple independently sampled outputs, we can *again* set the parameter `num_return_sequences > 1`:

In [ ]:
# set seed to reproduce results. Feel free to change the seed though to get different results
torch.random.manual_seed(0)

# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
sample_outputs = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_k=50,
    top_p=0.95,
    num_return_sequences=3
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: I enjoy walking with my cute dog or riding my bike with my friends as long as I can avoid a traffic jam. However, when I'm driving, I'm in control. If I have to pass, pass before I pass. If I have
1: I enjoy walking with my cute dog. I love my dog. She's my little friend who I love dearly.

I'll go with her when she has her lunch, but I've also done some errands and I know she loves a
2: I enjoy walking with my cute dog, but I would definitely encourage anyone who has a puppy to adopt a miniature and have her walk with him.

Why did you start making pictures of pets?

Well I wanted to make something from scratch


Cool, now you should have all the tools to let your model write your stories with `transformers`!

## Constrained generation

For better readability, we will use end-of-line as the EOS token, instead of just always generating 50 tokens.

In [12]:
END_OF_LINE = tokenizer('\n').input_ids[0]
print(END_OF_LINE)

198


The baseline beam search produces very similar sentences; all of them contain "not sure" or "don't think".

In [13]:
beam_outputs = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    no_repeat_ngram_size=2,
    num_return_sequences=5,
    early_stopping=True,
    eos_token_id=END_OF_LINE,
)

print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: I enjoy walking with my cute dog, but I don't have the time to do it every day.

1: I enjoy walking with my cute dog, but I don't have the time to do it every day. So, I decided to make my own dog food.

2: I enjoy walking with my cute dog, but I don't have the time to do it every day. So, I decided to make this recipe for you.

3: I enjoy walking with my cute dog, but I don't have the time to do it every day. So, I decided to make my own.

4: I enjoy walking with my cute dog, but I don't want to do it every day.



What would happen if we forbid the model to use these phrases?

In [14]:
beam_outputs = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    no_repeat_ngram_size=2,
    num_return_sequences=5,
    early_stopping=True,
    eos_token_id=END_OF_LINE,
    bad_words_ids=tokenizer(['sure', 'think', 'thundersnatch'], add_prefix_space=True)['input_ids'],
)

print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: I enjoy walking with my cute dog, but I don't want her to have to walk with me all the time. I want to be able to go to the park with her without having to worry about her getting hurt.

1: I enjoy walking with my cute dog, but I don't want her to have to walk with me all the time. I want to be able to go to the park with her without having to worry about her getting lost.

2: I enjoy walking with my cute dog, but I don't want her to have to walk with me all the time. I want to be able to go to the park with her without having to worry about her getting sick.

3: I enjoy walking with my cute dog, but I don't want her to have to walk with me all the time. I want to be able to go to the park with her without having to worry about her running away.

4: I enjoy walking with my cute dog, but I don't want her to have to walk with me all the time. I want to be able to take her out f

Why `add_prefix space`? Because the BPE tokenization used by GPT prepends the space to the next word, and this changes the token:

In [15]:
tokenizer(['sure', ' sure', ' I am not sure'])['input_ids']

[[19532], [1654], [314, 716, 407, 1654]]

We can see that the meaning of these texts has changed a lot – but in some unpredictable way.

Can we force the model to write a text involving cats?

In [16]:
beam_outputs = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    no_repeat_ngram_size=2,
    num_return_sequences=5,
    early_stopping=True,
    eos_token_id=END_OF_LINE,
    bad_words_ids=tokenizer(['sure', 'think'], add_prefix_space=True)['input_ids'],
    force_words_ids=[tokenizer(['cat'], add_prefix_space=True, add_special_tokens=False).input_ids],
)

print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: I enjoy walking with my cute dog, but I don't want her to have to walk with me all the time. I want to be able to go to the park with her and have her play with the other dogs. So I decided to make
1: I enjoy walking with my cute dog, but I don't want her to have to walk with me all the time. I want to be able to go to the park with her and have her play with the other dogs. So I decided to create
2: I enjoy walking with my cute dog, but I don't want her to have to walk with me all the time. I want to be able to go to the park with her and have her play with the other dogs. So I decided to build
3: I enjoy walking with my cute dog, but I don't want her to have to walk with me all the time. I want to be able to go to the park with her and have her play with the other dogs. So I decided to cat
4: I enjoy walking with my cute dog, but I don't want her to have to walk with me all

A clarification: **force_words_ids** is a list of constraints. Each constraint is a list of expressions, such that at least one expression should be included into the generated text. And each expression is just a list of tokens.

See the discussion in [the HF pull request](https://github.com/huggingface/transformers/issues/14081), or read the paper "[Guided Generation of Cause and Effect](https://www.ijcai.org/proceedings/2020/0502.pdf)" by Li et al, where the algorithm was proposed.  

To evaluate the power of these constraints, let us force the model to include a mouse (or even many mice) into the text. We can also relax the "cat" constraint by allowing the words "cats", "kitten" or "feline" instead.

In [17]:
beam_outputs = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    no_repeat_ngram_size=2,
    num_return_sequences=5,
    early_stopping=True,
    eos_token_id=END_OF_LINE,
    bad_words_ids=tokenizer(['sure', 'think'], add_prefix_space=True)['input_ids'],
    force_words_ids = [
        tokenizer(['cat', 'cats', 'kitten', 'feline', 'Cat', 'Cats'], add_prefix_space=True, add_special_tokens=False).input_ids,
        tokenizer(['mouse', 'mice'], add_prefix_space=True, add_special_tokens=False).input_ids,
    ],
)

print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: I enjoy walking with my cute dog. I love to play with her. She is so cute and I am so happy to have her with me. My husband and my daughter are very happy with the dog and the house. We are so grateful for
1: I enjoy walking with my cute dog. I love to play with her. She is so cute and I am so happy to have her with me. My husband and my daughter are very happy with the dog and the cat. They love the dogs and
2: I enjoy walking with my cute dog. I love to play with her. She is so cute and I am so happy to have her with me. My husband and my daughter are very happy with the dog and the house. We are so grateful f
3: I enjoy walking with my cute dog. I love to play with her. She is so cute and I am so happy to have her with me. My husband and my daughter are very happy with the dog and the cat. They love the dogs mouse
4: I enjoy walking with my cute dog. I love to play with he

## Use Logit Processor

In [23]:
from transformers import LogitsProcessor
import torch

class CustomLogitsProcessor(LogitsProcessor):
    def __init__(self, forbidden_tokens, penalty=1e6):
        """
        :param forbidden_tokens: List of token IDs to be penalized.
        :param penalty: Large negative value to suppress probability.
        """
        self.forbidden_tokens = forbidden_tokens
        self.penalty = penalty

    def __call__(self, input_ids, scores):
        """
        Modify logits before softmax is applied.
        :param input_ids: Tensor of tokenized input.
        :param scores: Tensor of shape (vocab_size,) representing logits.
        """
        for token in self.forbidden_tokens:
            scores[:, token] -= self.penalty  # Assign a very low probability
        return scores

custom_logit_processor = CustomLogitsProcessor(tokenizer(['sure', 'think', 'husband', 'daughter'], add_prefix_space=True)['input_ids'])

In [24]:
from transformers import AutoModelForCausalLM, AutoTokenizer, LogitsProcessorList


# Initialize the logits processor
logits_processors = LogitsProcessorList([custom_logit_processor])

beam_outputs = model.generate(
    input_ids,
    max_length=100,
    logits_processor=logits_processors,
    num_beams=5,
    no_repeat_ngram_size=2,
    num_return_sequences=5,
    early_stopping=True,
    eos_token_id=END_OF_LINE,
    force_words_ids = [
        tokenizer(['cat', 'cats', 'kitten', 'feline', 'Cat', 'Cats'], add_prefix_space=True, add_special_tokens=False).input_ids,
        tokenizer(['mouse', 'mice'], add_prefix_space=True, add_special_tokens=False).input_ids,
    ],
)

print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))



Output:
----------------------------------------------------------------------------------------------------
0: I enjoy walking with my cute dog. I love to play with her. She is so cute and I am so happy to have her with me. My dog is a very sweet and gentle dog and she is very loving. We have been together for over a year and a half and we are very happy. The only thing that is holding us back is the fact that we have not been able to find a good home for her yet. If you are looking for a dog that will be a great
1: I enjoy walking with my cute dog. I love to play with her. She is so cute and I am so happy to have her with me. My dog is a very sweet and gentle dog and she is very loving. We have been together for over a year and a half and we are very happy. The only thing that is holding us back is the fact that we have not been able to find a good home for her yet. If you are looking for a dog that will be a cat
2: I enjoy walking with my cute dog. I love to play with her. She is so

The texts satisfy the constraints and look fluent. Still, the model has somehow fooled us: it used the verb "to mouse" in its secondary sense, instead of referring to animals.

## **Conclusion**

As *ad-hoc* decoding methods, *top-p* and *top-K* sampling seem to produce more fluent text than traditional *greedy* - and *beam* search on open-ended language generation.
Recently, there has been more evidence though that the apparent flaws of *greedy* and *beam* search - mainly generating repetitive word sequences - are  caused by the model (especially the way the model is trained), rather than the decoding method, *cf.* [Welleck et al. (2019)](https://arxiv.org/pdf/1908.04319.pdf). Also, as demonstrated in [Welleck et al. (2020)](https://arxiv.org/abs/2002.02492), it looks as *top-K* and *top-p* sampling also suffer from generating repetitive word sequences.

In [Welleck et al. (2019)](https://arxiv.org/pdf/1908.04319.pdf), the authors show that according to human evaluations, *beam* search can generate more fluent text than *Top-p* sampling, when adapting the model's training objective.

Open-ended language generation is a rapidly evolving field of research and as it is often the case there is no one-size-fits-all method here, so one has to see what works best in one's specific use case.

Good thing, that *you* can try out all the different decoding methods in `transfomers` 🤗.

That was a short introduction on how to use different decoding methods in `transformers` and recent trends in open-ended language generation.

Feedback and questions are very welcome on the [Github repository](https://github.com/huggingface/transformers).

For more fun generating stories, please take a look at [Writing with Transformers](https://transformer.huggingface.co).

Thanks to everybody, who has contributed to the blog post: Alexander Rush, Julien Chaumand, Thomas Wolf, Victor Sanh, Sam Shleifer, Clément Delangue, Yacine Jernite, Oliver Åstrand and John de Wasseige.


## **Appendix**

There are a couple of additional parameters for the `generate` method that were not mentioned above. We will explain them here briefly!

- `min_length` can be used to force the model to not produce an EOS token (= not finish the sentence) before `min_length` is reached. This is used quite frequently in summarization, but can be useful in general if the user wants to have longer outputs.
- `repetition_penalty` can be used to penalize words that were already generated or belong to the context. It was first introduced by [Kesker et al. (2019)](https://arxiv.org/abs/1909.05858) and is also used in the training objective in [Welleck et al. (2019)](https://arxiv.org/pdf/1908.04319.pdf). It can be quite effective at preventing repetitions, but seems to be very sensitive to different models and use cases, *e.g.* see this [discussion](https://github.com/huggingface/transformers/pull/2303) on Github.

- `attention_mask` can be used to mask padded tokens
- `pad_token_id`, `bos_token_id`, `eos_token_id`: If the model does not have those tokens by default, the user can manually choose other token ids to represent them.

For more information please also look into the `generate` function [docstring](https://huggingface.co/transformers/main_classes/model.html?highlight=generate#transformers.TFPreTrainedModel.generate).